In [2]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data_sensors_path = '../CTC - Data Sample Sensors.csv'
data_sensors = pd.read_csv(data_sensors_path, delimiter=',')

# Convert Timestamp column to datetime
data_sensors['date'] = pd.to_datetime(data_sensors['date'], errors='coerce', utc=True)

In [6]:
data_bookings_path = '../CTC - Data Sample Bookings Groningen.csv'
data_bookings = pd.read_csv(data_bookings_path, delimiter=',')

# Convert Timestamp column to datetime
data_bookings['start time'] = pd.to_datetime(data_bookings['start time'], dayfirst=True, errors='coerce', utc=True)
data_bookings['end time'] = pd.to_datetime(data_bookings['end time'], dayfirst=True, errors='coerce', utc=True)

data_bookings

,reservation id,email,license plate,start time,end time,Smart Zone,City,Reservation Duration,Anonymous
0,NLGR10004L,producer@luthersbachensemble.nl,VD956V,2023-11-20 12:15:00+00:00,2023-11-20 12:45:00+00:00,Haddingestraat,Groningen,0:30:00,No
1,NLGR100108,producer@luthersbachensemble.nl,VD956V,2023-11-20 12:45:00+00:00,2023-11-20 13:15:00+00:00,Haddingestraat,Groningen,0:30:00,No
2,NLGR1002EF,producer@luthersbachensemble.nl,VD956V,2023-11-20 13:15:00+00:00,2023-11-20 13:45:00+00:00,Haddingestraat,Groningen,0:30:00,No
3,NLGR100390,producer@luthersbachensemble.nl,VD956V,2023-11-20 13:45:00+00:00,2023-11-20 14:15:00+00:00,Haddingestraat,Groningen,0:30:00,No
4,NLGR1004BC,producer@luthersbachensemble.nl,VD956V,2023-11-20 14:15:00+00:00,2023-11-20 14:45:00+00:00,Haddingestraat,Groningen,0:30:00,No
...,...,...,...,...,...,...,...,...,...
2330,NLGR995J5,producer@luthersbachensemble.nl,VD956V,2023-11-17 11:15:00+00:00,2023-11-17 11:45:00+00:00,Haddingestraat,Groningen,0:30:00,No
2331,NLGR9963D,producer@luthersbachensemble.nl,VD956V,2023-11-17 09:15:00+00:00,2023-11-17 09:30:00+00:00,Haddingestraat,Groningen,0:15:00,No
2332,NLGR997HI,vanderkamp@coupry.nl,98hjs9,2023-11-18 10:45:00+00:00,2023-11-18 11:00:00+00:00,Haddingestraat,Groningen,0:15:00,Yes
2333,NLGR998ED,tymenjanbronda@planet.nl,SR853V,2023-11-19 10:00:00+00:00,2023-11-19 10:30:00+00:00,Haddingestraat,Groningen,0:30:00,No
